# Circle detector exam project

### Python Projektformulering

#### Gruppe: Disgusted Obligation

#### Medlemmer: Jesper Hingelbjerg, Albert Scheel Vandel

Vores ide til et projekt er kameratracking, hvor man skal finde et objekt ud fra et, flere, fokuspunkter og bedømme afstanden, positionen og retningen af objektet, heriblandt når objektet bevæger sig, og sætte det ind i et koordinatsystem. Kameraet vil betragte objektet fra fugleperspektiv. Ideen er at tracke eksempelvis en, på objektet, optegnet cirkel med en viser, radiusstreg. Når cirklen deformeres eller på en eller anden vis skrumper, kan man således se at der er et skift i afstand. Hvis viseren peger i en anden retning, kan man se at der er tale om retningsskift. For at genkende cirklen med viseren vil vi benytte os af object recognition, og vi vil sikkert også bruge noget movement detection når objektet bevæger sig. Desuden kunne vi godt tænke os at mappe en sti ud fra hvor objektet hidtil har været. 

### Vores projekt delt op i tasks:
#### -Kunne finde en cirkel i kameraet.
#### -Kunne finde afstanden af en cirklen ud fra dens størrelse i kameraet.
#### -Kunne tegne en sti efter objektet.
#### -Kunne finde ud af i hvilken retning objektet bevæger sig ved at tracke positionen af objektet.
#### -Kunne finde objektets orientering ud fra viseren.
#### -Kombinere alt dette.


In [1]:
import numpy as np
import cv2
import sys
# I have measured the focal length of the webcam used, with the focalLengthCalc.ipynb notebook.
focalLength = 530
circleWidth = 10
video_capture = cv2.VideoCapture(0)
w, h = (1240, 720)
video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, w)
video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, h)
last_center_of_circle = [(0, 0)] * 10

while True:
    ret, frame = video_capture.read()
    if ret:
        grayRaw = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.Canny(grayRaw,75,150)
        rows = gray.shape[0]
        circles = cv2.HoughCircles(grayRaw,cv2.HOUGH_GRADIENT,1,rows/8,param1=300,param2=40,minRadius=10,maxRadius=300)
        
        if circles is not None:
            circles = np.uint16(np.around(circles))
            for i in circles[0, :]:
                center, radius = (i[0], i[1]), i[2]
                lines_x, lines_y = int(center[0] - radius), int(center[1] - radius)
                c_area = frame[lines_x:lines_x+(radius*2),lines_y:lines_y+(radius *2)]
                if c_area.shape[0] > 0 and c_area.shape[1] > 0:
                    c_color = cv2.resize(c_area, (1,1), interpolation=cv2.INTER_AREA)
                    c_color, c_thres = c_color[0][0], 90
                    if c_color[0] >= c_thres and c_color[1] >= c_thres and c_color[2] >= c_thres:
                        last_center_of_circle.append(center)

                        edges = cv2.Canny(frame,50,150,3)
                        lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength=100,maxLineGap=80)
                        if lines is not None:
                            for i in range(len(lines)):
    #                             if lines[i] is not None:                    
                                for x1,y1,x2,y2 in lines[i]:
                                    x_max, y_max = (lines_x+2*radius) + 10, (lines_y+2*radius) + 10
                                    if x1 >= lines_x and x2 >= lines_x and x1 <= x_max and x2 <= x_max and y1 >= lines_y and y2 >= lines_y and y1 <= y_max and y2 <= y_max:
                                            cv2.line(frame,(x1,y1),(x2,y2),(0,255,0),2)

                        sizeDistance = int((circleWidth * focalLength) / radius)
                        circleInfo = "CircleDistance = " + str(sizeDistance) + "cm"
                        cv2.putText(frame, circleInfo,(30,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)

                        cv2.circle(frame, center, 1, (0, 100, 100), 3)
                        cv2.circle(frame, center, radius, (255, 0, 255), 3)
                        if len(last_center_of_circle) >= 2:
                            x = [z[0] for z in last_center_of_circle]
                            y = [j[1] for j in last_center_of_circle]

                            for i in range (len(last_center_of_circle)-1):
                                start_x, start_y = int(x[i]), int(y[i])
                                finish_x, finish_y = int(x[i+1]), int(y[i+1])
                                if start_x > 0 and finish_x > 0:
                                    cv2.line(frame,(start_x, start_y),(finish_x, finish_y),(0,255,0),2)

                            last_center_of_circle.pop(0)

        cv2.imshow('Circle Detection Feed', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()